### This notebook achieves the following objectives:

    A) IDF-Vector Compute the IDF values for each word present in the corpus 
    of training samples (which in this case is the set of 3000 labeled companies)

#### Note: B and C updated to attempt to use scipy sparse matrix format. (old code still in notebook at end)
    B) Generate the graph. Compute the Jaccard similarity between 
    companies sharing similar high IDF words, and then populate the 
    graph where a node is a company and an edge is weighted by the 
    Jaccard similarity between the two companies (if calculated based on cutoff)

    C) Run a simple test of taking two companies, placing all of 
    their weighted edges in a respective vector, then computing 
    the dot product between these two edges

    Conclusion: We have found that the dot product method is 
    producing results in which the companies with the highest 
    dot products do in fact appear very similar. See part C for exact results


# A) IDF Vector
### To use an IDF vector, you have two options:

 #### Option 1: Use this code to generate a new IDF vector from words in the training dataset. This option has multiple cells to run
 
 #### Option 2: Use this code to read in a previously stored IDF vector. This option has one cell to run
The currently implemented option 1 use case reads in the entire 650,000 companies from the raw data file and filters down to 100,000 companies which all have full pitch book descriptions
             
 Note: Run either option 1 or option 2. If both are run, option 2 IDF vector will be used ebcause it will overwrite the variable set by option 1
 

  ----------------------------------------------------------------------

### Option 1

In [31]:
import math
import re
from operator import itemgetter
import string
import nltk

class TfIdf:

    """Tf-idf class implementing http://en.wikipedia.org/wiki/Tf-idf.
  
     The library constructs an IDF corpus and stopword list either from
     documents specified by the client, or by reading from input files.  It
     computes IDF for a specified term based on the corpus, or generates
     keywords ordered by tf-idf for a specified document.
    """

    def __init__(self, corpus_filename = None, stopword_filename = None,
               DEFAULT_IDF = 1.5):
        """Initialize the idf dictionary.  
    
        If a corpus file is supplied, reads the idf dictionary from it, in the
        format of:
        # of total documents
        term: # of documents containing the term

        If a stopword file is specified, reads the stopword list from it, in
        the format of one stopword per line.

        The DEFAULT_IDF value is returned when a query term is not found in the
        idf corpus.
        """
        self.num_docs = 0
        self.term_num_docs = {}     # term : num_docs_containing_term
        self.stopwords = []
        self.idf_default = DEFAULT_IDF

        if corpus_filename:
            corpus_file = open(corpus_filename, "r")

          # Load number of documents.
            line = corpus_file.readline()
            self.num_docs = int(line.strip())

          # Reads "term:frequency" from each subsequent line in the file.
            for line in corpus_file:
                tokens = line.split(":")
                term = tokens[0].strip()
                frequency = int(tokens[1].strip())
                self.term_num_docs[term] = frequency

        if stopword_filename:
            stopword_file = open(stopword_filename, "r")
            self.stopwords = [line.strip() for line in stopword_file]

    def get_tokens(self, doc):
        """Break a string into tokens, preserving URL tags as an entire token.

        This implementation does not preserve case.  
        Clients may wish to override this behavior with their own tokenization.
        """
        # Attempt 1 - Faster results (this one is faster than uncommented solution, but doesn't get rid of stop words)
        #str_list = [word.lower().translate(str.maketrans(' ', ' ', string.punctuation)) for word in re.split('\s|\.|-|,',str(doc))]
        
        punctuation = '[^\w\s]'
        doc = pd.Series(doc)
        txt = doc.str.lower().str.replace(punctuation, ' ').str.cat(sep=' ')
        stopwords = set(nltk.corpus.stopwords.words('english'))
        words = nltk.tokenize.word_tokenize(txt)
        return set(words) - stopwords

        
    def add_input_document(self, input):
        """Add terms in the specified document to the idf dictionary."""
        self.num_docs += 1
        words = set(self.get_tokens(input))
        for word in words:
            if word in self.term_num_docs:
                self.term_num_docs[word] += 1
            else:
                self.term_num_docs[word] = 1

    def save_corpus_to_file(self, idf_filename, stopword_filename,
                          STOPWORD_PERCENTAGE_THRESHOLD = 0.01):
        """Save the idf dictionary and stopword list to the specified file."""
        output_file = open(idf_filename, "w")

        output_file.write(str(self.num_docs) + "\n")
        for term, num_docs in self.term_num_docs.items():
            output_file.write(term + ": " + str(num_docs) + "\n")

        sorted_terms = sorted(self.term_num_docs.items(), key=itemgetter(1),
                          reverse=True)
        stopword_file = open(stopword_filename, "w")
        for term, num_docs in sorted_terms:
            if num_docs < STOPWORD_PERCENTAGE_THRESHOLD * self.num_docs:
                break

            stopword_file.write(term + "\n")

    def get_num_docs(self):
        """Return the total number of documents in the IDF corpus."""
        return self.num_docs

    def get_idf(self, term):
        """Retrieve the IDF for the specified term. 
    
        This is computed by taking the logarithm of ( 
        (number of documents in corpus) divided by (number of documents
        containing this term) ).
        """
        if term in self.stopwords:
            return 0

        if not term in self.term_num_docs:
            return self.idf_default

        return math.log(float(1 + self.get_num_docs()) / (1 + self.term_num_docs[term]))

#### Now read in the full raw dataset

In [8]:
import pandas as pd
import numpy as np

#################### Full Data #########################
training_categories_df = pd.read_csv("../../data/raw_data_fixed.csv",  encoding = "ISO-8859-1", usecols=['domain'\
, 'tx_industry', 'cb_category', 'tx_category', 'cb_desc', 'pb_desc', 'pb_category'])

#mydoclist = training_categories_df.ix[0:,'pb_desc'].values

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Filter out companies that do not have a pitch book provided description

In [9]:

# Filter out all companies that don't have a pitch book provided description
subset_df_pb_desc = training_categories_df.ix[training_categories_df['pb_desc'].notnull()]


# Now print out some info
len_of_df = subset_df_pb_desc.shape[0]
print("There are {} companies that have a full pitch book provided description".format(len_of_df))


There are 166406 companies that have a full pitch book provided description


In [10]:
# Cut this down to 100,000 companies using the head function
subset_df_pb_desc = subset_df_pb_desc.head(100000)
print("Size of final dataframe to use for building the graph: {}".format(subset_df_pb_desc.shape[0]))
#subset_df_pb_desc

Size of final dataframe to use for building the graph: 100000


In [11]:
# Uncomment below line if you need to write the filtered companies to a csv
subset_df_pb_desc.to_csv("../../data/100000_companies_with_description.csv")

In [28]:
mydoclist = subset_df_pb_desc['pb_desc'].values
print(len(mydoclist))


100000


#### This code computes the actual IDF vector and can take a couple minutes to run with 100,000 samples

In [29]:
import numpy as np

idfcalc = TfIdf()
for entry in mydoclist:
    idfcalc.add_input_document(entry)
#print(idfcalc.term_num_docs)

idf_vec = []
term_vec = []
for term in idfcalc.term_num_docs:
    idf = idfcalc.get_idf(term)
    idf_vec.append(idf)
    term_vec.append(term)

#### Convert the IDF vector a pandas Series and sort by IDF value (in descending order)

In [34]:
idf_vector = pd.Series(idf_vec, index=term_vec)

sorted_idf_vector = idf_vector.sort_values(ascending=False)
idf_vector = sorted_idf_vector
print(idf_vector)

dioxin               10.819788
spiegelmers          10.819788
nanopositioning      10.819788
schlumberger         10.819788
handelsblad          10.819788
nrc                  10.819788
gassification        10.819788
pyro                 10.819788
tme                  10.819788
ntag                 10.819788
latticed             10.819788
ntp                  10.819788
neurotherapeutics    10.819788
trickster            10.819788
pangya               10.819788
omnigen              10.819788
nutricosmetics       10.819788
acreages             10.819788
protide              10.819788
characteristic       10.819788
epicheck             10.819788
careful              10.819788
chemetics            10.819788
photosensitive       10.819788
suscription          10.819788
nanopositioners      10.819788
cauterization        10.819788
miltiple             10.819788
colli                10.819788
quadrantanopia       10.819788
                       ...    
allows                2.749195
informat

Uncomment the below line to save the IDF vector to a file

In [35]:
# This code saves the IDF vector to a file
# idf_vector.to_csv("../../data/100000_companies_with_description_idf_vector.csv")

#### End Option 1

### Option 2 (skip this if option 1 was utilized, else proceed to read an IDF vector from a csv file)

In [58]:
import pandas as pd
import numpy as np

### Read in the IDF vector  - 100000 companies
#idf_vector = pd.read_csv("../../data/100000_companies_with_description_idf_vector.csv" ,header=None, \
#                         names=['IDF'], index_col=0, encoding = "ISO-8859-1")

### Read the IDF vector - 3000 companies
idf_vector = pd.read_csv("../../data/training_idf_vector.csv" ,header=None, names=['IDF'], index_col=0, encoding = "ISO-8859-1")
idf_vector

,IDF
3,7.313554
cranberries,7.313554
siteÃ­s,7.313554
maintainable,7.313554
fermented,7.313554
raspberry,7.313554
pomegranate,7.313554
juice,7.313554
chinesespeaking,7.313554
loop,7.313554


#### End Option 2

#### Run the next code no matter which option was used

In [59]:

# Get the idf values in a column vector
idf_values = list(idf_vector.values)

# Get the words in a column vector. The initial order mathes the 
# values in the idf_values_array
idf_words = list(idf_vector.index.values)
# Perform a reshape on the words array to get it in a better format

idf_set = set(idf_words)
idf_map = dict(zip(idf_words, idf_values))


# B) Creating the Graph  (Updated to try to use Sparse matrix)

The next step will be to create an adjacency matrix to store all these values.

In [60]:
import pandas as pd

##### 100,0000 companies
#company_list = pd.read_csv('../../data/100000_companies_with_description.csv',  encoding = "ISO-8859-1", \
#                           usecols=['domain', 'tx_industry', 'cb_category', 'tx_category', 'cb_desc',\
#                                    'pb_desc', 'pb_category'])

##### 3000 companies
company_list = pd.read_csv('../../data/category_training_labeled_fixed.csv', encoding = "ISO-8859-1")

company_list.head()

,domain,rs_category,tx_industry,cb_category,tx_category,pb_desc,cb_desc,pb_industry,pb_category
0,conferencecloud.co,Business Communication Application,NaN,information technology,NaN,Provider of an online conferencing platform. T...,ConferenceCloud provides state-of-the-art live...,NaN,Social/Platform Software
1,terminus.com,Marketing Software and Service,"AdTech,AdTech,Enterprise Applications",advertising|advertising platforms|b2b,"AdTech,MarketingTech,SaaS",Developer of a B2B advertising platform. The c...,Terminus is a platform that seamlessly integra...,"AdTech, Marketing Tech",Business/Productivity Software
2,galileoprocessing.com,Payment Application,Fintech,NaN,Payment Cards,Provider of payment processing services. The c...,Next generation card processing platform,FinTech,Other Financial Services
3,pubble.co,Business Communication Application,NaN,digital media|education|software,NaN,Operator of a community engagement platform. T...,Pubble is a messaging platform that simplifies...,NaN,Software Development Applications
4,cajo.fi,Printing Technology,NaN,NaN,NaN,Developer of a stainless steel colour patterni...,NaN,Manufacturing,Machinery (B2B)


## ============ Attempt to use a sparse matrix

In [61]:
import numpy as np
from scipy.sparse import lil_matrix

n_companies = company_list.shape[0]
company_graph = lil_matrix((n_companies,n_companies))
#company_graph[:] = -1
company_graph

<3000x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in LInked List format>

In [62]:
import string
import nltk
nltk.download('punkt')

#Gets the words out of the labeled descriptions
def get_words(df):
    punctuation = '[^\w\s]'
    txt = df.str.lower().str.replace(punctuation, ' ').str.cat(sep=' ')
    stopwords = set(nltk.corpus.stopwords.words('english'))
    words = nltk.tokenize.word_tokenize(txt)
    return set(words) - stopwords

[nltk_data] Downloading package punkt to /Users/Connor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
company_words_list = [set()]*len(company_list)
for i in range(len(company_list)):
    #start_index = 5
    #end_index = 7
    if i%10000 is 0:
        print("iteration {}".format(i))
    company_words = get_words(company_list.iloc[i,5:6])
    company_words_list[i] = company_words

iteration 0


In [66]:
#given a target word and a pandas data frame of companies, 
# returns a list of companies whose descriptions contain the target word
def get_companies(target_word, company_words_list):
    candidate_set = set()
    for i in range(len(company_words_list)):
        company_description = company_words_list[i]
        if target_word in company_description:
            candidate_set.add(i)
    return list(candidate_set)

In [67]:
def get_similarity(company_index_1, company_index_2, company_words_list, idf_set, idf_map):
    company_1 = company_words_list[company_index_1]
    company_2 = company_words_list[company_index_2]
    intersection = company_1 & company_2
    union = company_1 | company_2
    if len(union) == 0:
        return 0
    intersection_score = 0.0
    union_score = 0.0
    for word in union:
        if word in idf_set:
            word_score = idf_map[word][0]
            union_score += word_score
            if word in intersection:
                intersection_score += word_score
                
    return intersection_score/union_score

### Below is an attempt at buliding the graph into a scipy sparse lil_matrix format, with a slight modification to try and help scale the code
   It appeared that once we found a pair of companies that met the similarity cutoff, we added the value to the (i,k) position, and to the (k,i) position. From how I understood it, we are then looking at the same position over again later on in the iterations and computing the same value twice. I therefore attempted to add a dictionary look up table, and I added the (k,i) position, convereted to a string, as a key. I then check if that is in the dictionary every kth iteration, and if it is, I skip that iteration (since we already calculated and stored the value in both positions). Still think it might be a little buggy because it doesn't seem to speed up like I think it should once the count of (n_companies - len(visited_table)) is zero. I added a break statement to break the outer loop once this occurs.

Note: The lil_matrix way to access an index is via "lil_matrix[i,k]" instead of "matrix[i][k]"

In [69]:
n_updated_elements = 0
n_companies = len(company_list)
cutoff = 0.1

#Seems like this code is redundant, in that it 
visited_table = {}

for i in range(n_companies):
    if n_companies - len(visited_table) <= 0:
        print("Finished")
        break
    for k in range((i+1), n_companies):
        if str(k)+str(i) in visited_table or str(i)+str(k) in visited_table:
            continue
        else:
            edge_weight = get_similarity(i, k, company_words_list, idf_set, idf_map)
            if edge_weight >= cutoff:
                company_graph[i,k] = edge_weight
                company_graph[k,i] = edge_weight
            # Add these companies to graph and skip later on to save time
                visited_table[str(k)+str(i)] = 1
    if i%10 is 0:
        print("ith iteration: {} out of {}".format(i,n_companies - len(visited_table)))


ith iteration: 0 out of 2976
ith iteration: 10 out of 2925
ith iteration: 20 out of 2846
ith iteration: 30 out of 2741
ith iteration: 40 out of 2682
ith iteration: 50 out of 2605
ith iteration: 60 out of 2446
ith iteration: 70 out of 2357
ith iteration: 80 out of 2261
ith iteration: 90 out of 2220
ith iteration: 100 out of 2206
ith iteration: 110 out of 2144
ith iteration: 120 out of 2019
ith iteration: 130 out of 1929
ith iteration: 140 out of 1885
ith iteration: 150 out of 1846
ith iteration: 160 out of 1831
ith iteration: 170 out of 1764
ith iteration: 180 out of 1657
ith iteration: 190 out of 1518
ith iteration: 200 out of 1340
ith iteration: 210 out of 1263
ith iteration: 220 out of 1226
ith iteration: 230 out of 1114
ith iteration: 240 out of 1001
ith iteration: 250 out of 880
ith iteration: 260 out of 653
ith iteration: 270 out of 556
ith iteration: 280 out of 421
ith iteration: 290 out of 320
ith iteration: 300 out of 300
ith iteration: 310 out of 222
ith iteration: 320 out of 

### Not sure about this code below either, but the setting values < 0 to 0 appears to run. Not sure if it works as intended though because the sparse matrix format doesn't work with alll normal numpy operations

In [81]:
# np.fill_diagonal(company_graph, 1)
company_graph[company_graph < 0] = 0

### Didn't make any changes to the next line

In [72]:
cutoff_sparsity = np.zeros(20)
#cutoff_sparsity[i] gives the sparsity of the graph if similarity threshold is (i+1)*0.01
n_possible_edges = float(company_graph.size - n_companies)
for i in range(20):
    cutoff_sparsity[i] = \
    (company_graph[company_graph > (i+1)*0.01].size - n_companies)/n_possible_edges
print(cutoff_sparsity)

[ 1.          1.          1.          1.          1.          1.          1.
  1.          1.          1.          0.54254302  0.23924474  0.0583174
 -0.07194073 -0.15941683 -0.21510516 -0.25740918 -0.28680688 -0.30544933
 -0.31883365]


In [73]:
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import pylab
G=nx.Graph()
G.add_nodes_from(company_list['domain'].values)
#G.add_nodes_from([x in range(n_companies)])
for i in range(n_companies):
    for k in range(i, n_companies):
        if company_graph[i,k] != 0:         # changed this indexing operation to [i,k] from [i][k]
            G.add_edge(i,k)
            G[i][k]['weight'] = company_graph[i,k]


def save_graph(graph,file_name):
    #initialze Figure
    plt.figure(num=None, figsize=(200, 200), dpi=80)
    plt.axis('off')
    fig = plt.figure(1)
    #pos = nx.spring_layout(graph)
    pos = nx.spring_layout(graph,k=0.9,iterations=5)
    nx.draw_networkx_nodes(graph,pos,node_color='g',node_size = 30,linewidths=0)
    nx.draw_networkx_edges(graph,pos,edge_color='b')
    nx.draw_networkx_labels(graph,pos,label_size = 30)

    cut = 1.00
    xmax = cut * max(xx for xx, yy in pos.values())
    ymax = cut * max(yy for xx, yy in pos.values())
    plt.xlim(0, xmax)
    plt.ylim(0, ymax)

    plt.savefig(file_name,bbox_inches="tight")
    pylab.close()
    del fig


save_graph(G,"3K_graph_sparse_matrix_version.pdf")

# C) New Dot Product Heuristic Attempt

### Now convert the lil_matrix to a csr_matrix format and do the dot products

- why? from scipy documentation, csr_matrix format is more efficient for doing matrix multiplication, while the lil_matrix format is better for fast indexing/building the matrix

In [76]:
from scipy.sparse import csr_matrix

company_graph_csr = company_graph.tocsr()
company_graph_csr

<3000x3000 sparse matrix of type '<class 'numpy.float64'>'
	with 11368 stored elements in Compressed Sparse Row format>

### Having trouble getting the code from here on out working

In [78]:
# Not sure what this is for, so having trouble getting it to work with csr_matrix format
mask = ~np.eye(company_graph_csr.shape[0], dtype = bool)
extremes = np.where((mask) & (company_graph_csr > 0.3))

TypeError: unsupported operand type(s) for &: 'bool' and 'csr_matrix'

In [79]:
dot_products = np.zeros(100)
for i in range(len(dot_products)):
    company_1 = extremes[0][i]
    company_2 = extremes[1][i]
    dot_products[i] = company_graph_csr[company_1].dot(company_graph_csr[company_2])
dot_products

NameError: name 'extremes' is not defined

In [ ]:
sorted_scores = np.sort(dot_products)
sorted_scores = sorted_scores[::-1]
for i in range(5):
    print('Highest Score Pair ' + str(i + 1))
    print('--------------------------')
    score = sorted_scores[2*i]
    index = np.where(dot_products == score)
    company_1 = extremes[0][index[0][0]]
    company_2 = extremes[1][index[0][0]]
    
    print("Company 1: {}\nCompany 2: {}\n".format(company_list.ix[company_1][0], company_list.ix[company_2][0]))

    print("Company 1 Description 1:\n{}\n".format(company_list.ix[company_1][5]))
    print("Company 2 Description 1:\n{}".format(company_list.ix[company_2][5]))

    print('\n')
    print("Company 1 Description 2:\n{}\n".format(company_list.ix[company_1][6]))
    print("Company 2 Description 2:\n{}".format(company_list.ix[company_2][6]))
    print('\n\n')

In [ ]:
sorted_scores = np.sort(dot_products)
for i in range(5):
    print('Lowest Score Pair ' + str(i + 1))
    print('--------------------------')
    score = sorted_scores[2*i]
    index = np.where(dot_products == score)
    company_1 = extremes[0][index[0][0]]
    company_2 = extremes[1][index[0][0]]
    
    print("Company 1: {}\nCompany 2: {}\n".format(company_list.ix[company_1][0], company_list.ix[company_2][0]))

    print("Company 1 Description 1:\n{}\n".format(company_list.ix[company_1][5]))
    print("Company 2 Description 1:\n{}".format(company_list.ix[company_2][5]))

    print('\n')
    print("Company 1 Description 2:\n{}\n".format(company_list.ix[company_1][6]))
    print("Company 2 Description 2:\n{}".format(company_list.ix[company_2][6]))
    print('\n\n')

# =============== End of new code/ Start of old graph building code

In [ ]:
n_companies = company_list.shape[0]
company_graph = np.empty((n_companies,n_companies))
company_graph[:] = -1
company_graph

Next, we'll go through each word and see which companies have that word. 
We'll go through the top 1000 idf words, find the companies with those words, 
and then compare them to create the edge weight in the graph.  
First, the following function creates a set of words out of the description

In [15]:
import string
import nltk
nltk.download('punkt')

#Gets the words out of the labeled descriptions
def get_words(df):
    punctuation = '[^\w\s]'
    txt = df.str.lower().str.replace(punctuation, ' ').str.cat(sep=' ')
    stopwords = set(nltk.corpus.stopwords.words('english'))
    words = nltk.tokenize.word_tokenize(txt)
    return set(words) - stopwords



[nltk_data] Downloading package punkt to /Users/Connor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Now, we create a company_words_list, i.e. each company is associated with a set of words.

In [16]:
company_words_list = [set()]*len(company_list)
for i in range(len(company_list)):
    start_index = 5
    end_index = 7
    company_words = get_words(company_list.iloc[i,start_index:end_index])
    company_words_list[i] = company_words


This function goes through the list of companies and sees if a 
given word is in the description for each of the companies, 
returning a set of company indices with that word.

In [17]:
#given a target word and a pandas data frame of companies, 
# returns a list of companies whose descriptions contain the target word
def get_companies(target_word, company_words_list):
    candidate_set = set()
    for i in range(len(company_words_list)):
        company_description = company_words_list[i]
        if target_word in company_description:
            candidate_set.add(i)
    return list(candidate_set)
        

This function takes a candidate pair, and computes their weighted 
similarity by finding Jaccard similarity and then weighing it by the idf of the words.

In [18]:
def get_similarity(company_index_1, company_index_2, company_words_list, idf_set, idf_map):
    company_1 = company_words_list[company_index_1]
    company_2 = company_words_list[company_index_2]
    intersection = company_1 & company_2
    union = company_1 | company_2
    if len(union) == 0:
        return 0
    intersection_score = 0.0
    union_score = 0.0
    for word in union:
        if word in idf_set:
            word_score = idf_map[word][0]
            union_score += word_score
            if word in intersection:
                intersection_score += word_score
                
    return intersection_score/union_score
        

Now, we go through and construct the 3000x3000 adjacency matrix.

In [19]:
n_updated_elements = 0
n_companies = len(company_list)
cutoff = 0.1
for i in range(n_companies):
    for k in range((i+1), n_companies):
        edge_weight = get_similarity(i, k, company_words_list, idf_set, idf_map)
        if edge_weight >= cutoff:
            company_graph[i][k] = edge_weight
            company_graph[k][i] = edge_weight
        
#removing -1's and ensuring 1's along the diagonal

np.fill_diagonal(company_graph, 1)
company_graph[company_graph < 0] = 0

Here I test different cutoffs for similarity scores to count as edges.

In [21]:
cutoff_sparsity = np.zeros(20)
#cutoff_sparsity[i] gives the sparsity of the graph if similarity threshold is (i+1)*0.01
n_possible_edges = float(company_graph.size - n_companies)
for i in range(20):
    cutoff_sparsity[i] = \
    (company_graph[company_graph > (i+1)*0.01].size - n_companies)/n_possible_edges
print(cutoff_sparsity)

[  2.96565522e-03   2.96565522e-03   2.96565522e-03   2.96565522e-03
   2.96565522e-03   2.96565522e-03   2.96565522e-03   2.96565522e-03
   2.96565522e-03   2.96565522e-03   1.90018895e-03   1.23107703e-03
   8.17161276e-04   5.47960431e-04   3.82127376e-04   2.66310993e-04
   1.89841058e-04   1.33822385e-04   9.80326776e-05   7.44692675e-05]


# Dot Product Heuristic

Now I'm going to perform a very simple heuristic to see how 
well the dot products predict similarity by seeing how they 
function on the company pairs with the highest textual similarity scores.

In [22]:
mask = ~np.eye(company_graph.shape[0], dtype = bool)
extremes = np.where((mask) & (company_graph > 0.3))

In [23]:
dot_products = np.zeros(len(extremes[0]))
for i in range(len(dot_products)):
    company_1 = extremes[0][i]
    company_2 = extremes[1][i]
    dot_products[i] = company_graph[company_1].dot(company_graph[company_2])
dot_products

array([ 0.69330862,  0.90417218,  1.71394821,  1.61786232,  1.29913586,
        1.2257629 ,  0.95953763,  0.95953763,  1.05905751,  0.85415079,
        0.8595902 ,  1.05905751,  0.8595902 ,  0.94622351,  0.81868064,
        1.2257629 ,  0.90417218,  0.85326372,  0.85415079,  1.71394821,
        1.19521275,  0.94622351,  0.76862511,  0.81868064,  0.75469454,
        1.08860499,  0.69330862,  0.75469454,  1.26190621,  1.61786232,
        0.76862511,  1.08860499,  1.26190621,  0.85326372,  1.29913586,
        1.19521275])

Now I'll look at the descriptions for the 5 highest dot products.

In [40]:
sorted_scores = np.sort(dot_products)
sorted_scores = sorted_scores[::-1]
for i in range(5):
    print('Highest Score Pair ' + str(i + 1))
    print('--------------------------')
    score = sorted_scores[2*i]
    index = np.where(dot_products == score)
    company_1 = extremes[0][index[0][0]]
    company_2 = extremes[1][index[0][0]]
    
    print("Company 1: {}\nCompany 2: {}\n".format(company_list.ix[company_1][0], company_list.ix[company_2][0]))

    print("Company 1 Description 1:\n{}\n".format(company_list.ix[company_1][5]))
    print("Company 2 Description 1:\n{}".format(company_list.ix[company_2][5]))

    print('\n')
    print("Company 1 Description 2:\n{}\n".format(company_list.ix[company_1][6]))
    print("Company 2 Description 2:\n{}".format(company_list.ix[company_2][6]))
    print('\n\n')

Highest Score Pair 1
--------------------------
Company 1: csb-bk.com
Company 2: cincinnatifederal.com

Company 1 Description 1:
Operator of a bank holding company. The company through its subsidiary provides personal and commercial banking services to its customers.

Company 2 Description 1:
Operator of a bank holding company. The company through its subsidiaries provides banking and banking and financial services to individual and corporate customers.


Company 1 Description 2:
nan

Company 2 Description 2:
nan



Highest Score Pair 2
--------------------------
Company 1: groffr.com
Company 2: pocketlistings.net

Company 1 Description 1:
nan

Company 2 Description 1:
nan


Company 1 Description 2:
groffr.com is real estate based company.

Company 2 Description 2:
Off Market Real Estate Network



Highest Score Pair 3
--------------------------
Company 1: ideaforge.co.in
Company 2: canarddrones.com

Company 1 Description 1:
Developer of drones. The company develops and manufactures au

Now let's look at the 5 lowest dot products out of the pairs with the highest similarities.

In [5]:
sorted_scores = np.sort(dot_products)
for i in range(5):
    print('Lowest Score Pair ' + str(i + 1))
    print('--------------------------')
    score = sorted_scores[2*i]
    index = np.where(dot_products == score)
    company_1 = extremes[0][index[0][0]]
    company_2 = extremes[1][index[0][0]]
    
    print("Company 1: {}\nCompany 2: {}\n".format(company_list.ix[company_1][0], company_list.ix[company_2][0]))

    print("Company 1 Description 1:\n{}\n".format(company_list.ix[company_1][5]))
    print("Company 2 Description 1:\n{}".format(company_list.ix[company_2][5]))

    print('\n')
    print("Company 1 Description 2:\n{}\n".format(company_list.ix[company_1][6]))
    print("Company 2 Description 2:\n{}".format(company_list.ix[company_2][6]))
    print('\n\n')

NameError: name 'dot_products' is not defined

    Both the top 5 and bottom 5 pairs had very high similarity for the most part, but perhaps that's just because I looked at such an extreme top end of textual similarity scores.  However, even here, the de-noising via dot product seems to work, especially in the case of homepage.com and jupviec.vn, which aren't as similar as their high textual similarity would seem to suggest.

## Connected Components

Now I'll check how connected the graph is.

In [36]:
import networkx as nx
G=nx.Graph()
G.add_nodes_from(company_list.index.values)
for i in range(n_companies):
    for k in range(i+1, n_companies):
        if company_graph[i][k] != 0:
            G.add_edge(i,k)
            G[i][k]['weight'] = company_graph[i][k]

print("Is the graph fully connected?: {}".format(nx.is_connected(G)))
print("Number of nodes in the graph: {}".format(nx.number_of_nodes(G)))
print("Number of connected components: {}".format(nx.number_connected_components(G)))

Is the graph fully connected?: False
Number of nodes in the graph: 3000
Number of connected components: 494


    The graph isn't connected, and has many connected components.  Let's see how nodes are distributed across these components.

In [181]:
connected_components = sorted(nx.connected_components(G), key = len, reverse = True)
n_components = len(connected_components)
elements_per_component = np.empty(n_components, dtype = int)
for i in range(n_components):
    elements_per_component[i] = len(connected_components[i])
elements_per_component


array([2499,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,   

    The majority of nodes are in one connected component, and the rest of the connected components are almost exclusively single nodes with no edges, with the exception of a few lone pairs.  Thus, while the graph isn't connected, for our intents/purposes we can treat it like it is (nodes with no edges won't provide any value in determining similarity), allowing the dot product/shared neighbors approach to potentially work.

## Conclusion

    Taking dot products seems to provide for some level of correction (albeit based on 1 pair that was marginally corrected by the low dot-product), even in the case of the pairs with the highest textual similarity.  It thus seems like a good denoising technique to use on the real graph.